<a href="https://colab.research.google.com/github/jamesdvance/video_code/blob/master/AutoEncoder_LSTM_JV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving test.csv to test.csv
Saving train_ss.csv to train_ss.csv


In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df = df.sort_values(['host','process','timestamp']).reset_index()

In [ ]:
num_cols = [col for col in df.columns.values if col not in ['count_col', 'isAnomaly','host', 'process','timestamp']]

In [ ]:
data =df[num_cols].values

LSTM

In [ ]:
input_data = data.reshape(data.shape[0],1,data.shape[1])

In [ ]:
def autoencoder_model(X):
    """ Takes a matrix """
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True,
             kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu',return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu',return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
model = autoencoder_model(input_data)

In [ ]:
model.compile(optimizer='adam', loss='mae')

In [ ]:
nb_epochs = 50
batch_size=10
history =model.fit(input_data, 
                    input_data, 
                    epochs=nb_epochs, 
                    batch_size=batch_size,
                   validation_split=0.05).history

In [ ]:
x_pred_df = pd.DataFrame(x_pred.reshape(x_pred.shape[0],
                                       x_pred.shape[2]))

In [ ]:
x_pred_df.index = train_df.index

In [ ]:
scored = pd.DataFrame(index=train_df.index)

In [ ]:
input_reshaped = input_data.reshape(input_data.shape[0],input_data.shape[2])

In [ ]:
scored['loss_mae'] = np.mean(np.abs(x_pred_df.values-train_df[num_cols].values),axis=1)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(16,9), dpi=80)
plt.title("Loss Distribution")
sns.distplot(scored['loss_mae'],bins=20, kde=True, color='blue')
plt.xlim([0.0,.13])

In [ ]:
test_df = pd.read_csv(datapath+"test.csv")

In [ ]:
x_test = test_df[num_cols].values

In [ ]:
input_test = x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [ ]:
x_pred = model.predict(input_test)

In [ ]:
test_pred_df =pd.DataFrame(x_pred.reshape(x_pred.shape[0],
                                       x_pred.shape[2]))

In [ ]:
test_pred_df.index = test_df.index

In [ ]:
scored = pd.DataFrame(index=test_df.index)

In [ ]:
scored['loss_mae'] = np.mean(np.abs(test_pred_df.values-test_df[num_cols].values),axis=1)

In [ ]:
scored['Anomaly'] = scored['loss_mae'] >= 0.1

In [ ]:
scored.to_csv("lstm_pred.csv")

In [ ]:
files.download("lstm_pred.csv")

In [ ]:
plt.figure(figsize=(16,9), dpi=80)
plt.title("Loss Distribution")
sns.distplot(scored['loss_mae'],bins=20, kde=True, color='blue')
plt.xlim([0.0,.13])